In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import dnasufo.dnasufo as dnasufo
shape = [50, 60, 60]
img = np.zeros(shape)
t = np.arange(shape[0])
angle = np.random.uniform(0, np.pi)
center = np.array([shape[-2], shape[-1]]).reshape((2,1))
delta = (0.5 + np.arctan(t - shape[0] / 4)/np.pi)
p1 = np.stack((t, shape[1] * (0.5 + np.cos(angle)*delta / 5),  shape[2] * (0.5 + np.sin(angle)*delta / 5))).astype(int)
p2 = np.stack((t, center[0] * (0.5 + np.cos(-angle)*delta / 5),  center[1] * (0.5 + np.sin(-angle)*delta / 5))).astype(int)
img[tuple(p1)] = 1
img[tuple(p2)] = 1
img1 = ndi.gaussian_filter(img, [0,4,4])
img2 = (ndi.gaussian_filter(img, [0,3,3]) > 0.0001) & (ndi.gaussian_filter(img, [0,3,3]) < 0.0005)
img2 = ndi.gaussian_filter(img2.astype(float), [0,2,2])
# plt.imshow(img2[9])
data = np.stack((img1/img1.max(),img2/img2.max()),1)
data = 100 + 50 * data / data.max()
data = data + np.random.normal(0,5,data.shape)
plt.imshow(dnasufo.uv2rgb(data[25]))

In [ ]:
import numpy as np
from scipy import ndimage as ndi
shape = [50,2,60,60]
img = np.array(shape)
for frame in range(shape[0]):
    if frame < 0.5 * shape[0]:



In [ ]:
import napari
napari.view_image(data,channel_axis=1)

In [ ]:
import dnasufo.dnasufo as mq
pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow = mq.process(data)

In [ ]:
import napari
v = napari.view_image(data,channel_axis=1)
v.add_labels(cell_lbl.squeeze())

In [ ]:
mq.create_figure(0,"test",pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow,10)

In [ ]:
from pathlib import Path
import tifffile
import dnasufo.dnasufo as mq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# list files from the src folder
root = Path('/media/cephfs2/jparham/')
dst = Path('/media/cephfs2/jeromeb/userdata/Baum_group/jparham/Analysis7/')
filelist = pd.read_csv(dst/'filelist.csv')
filelist = filelist.replace(u"\uf028"," ")

# 
index = 0
filename = filelist['path'].iloc[index]
print(filename)
pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow = mq.process(filename)
plt.imshow(mq.uv2rgb(pimg[0]))
plt.plot(cell_trj['centroid-1'], cell_trj['centroid-0'])
plt.title(Path(filename).stem)

In [ ]:
plt.imshow(pimg[100,0])

In [ ]:
import napari
v = napari.view_image(pimg,channel_axis=1)
v.add_labels(np.squeeze(cell_lbl))
v.add_labels(np.squeeze(dna_lbl))
v.add_tracks(cell_trj[['label', 'frame','centroid-0','centroid-1']])

In [ ]:
mq.create_figure(index, filename, pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow, frame=100)

In [ ]:
mq.create_strip(filename, pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow,selection=slice(0,200,20))

In [ ]:
results_path = Path('tmp.h5')
if results_path.exists():
    results_path.unlink()
mq.save_result(results_path, Path(filename).stem, pimg, cell_lbl, cell_trj, cell_flow, dna_lbl, dna_trj, dna_flow)